In [39]:
import os
os.chdir(r'C:\Users\User\Desktop\AI-ML\Project 1\Anamoly_detec_data')
import sys
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from scipy.io import loadmat


### Import PYod packages and methods

In [40]:
from pyod.models.pca import PCA
from pyod.models.mcd import MCD
from pyod.models.ocsvm import OCSVM
from pyod.models.lof import LOF
from pyod.models.cblof import CBLOF
from pyod.models.knn import KNN
from pyod.models.hbos import HBOS
from pyod.models.abod import ABOD
from pyod.models.feature_bagging import FeatureBagging
from pyod.models.iforest import IForest

### Import Metrics Packages

In [41]:
from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score

### Define data file and read X and Y

In [42]:
mat_file_list = ['arrhythmia.mat','cardio.mat','glass.mat',
                 'ionosphere.mat','letter.mat','lympho.mat'
                 ,'mnist.mat','musk.mat','optdigits.mat',
                 'pendigits.mat','pima.mat','satellite.mat'
                 ,'satimage-2.mat','shuttle.mat',
                 'shuttle.mat','vertebral.mat',
                 'vowels.mat','wbc.mat']

In [43]:
mat_file_list

['arrhythmia.mat',
 'cardio.mat',
 'glass.mat',
 'ionosphere.mat',
 'letter.mat',
 'lympho.mat',
 'mnist.mat',
 'musk.mat',
 'optdigits.mat',
 'pendigits.mat',
 'pima.mat',
 'satellite.mat',
 'satimage-2.mat',
 'shuttle.mat',
 'shuttle.mat',
 'vertebral.mat',
 'vowels.mat',
 'wbc.mat']

### Loading mat file

In [44]:


data=loadmat('cardio.mat')

In [45]:
data

{'__header__': b'MATLAB 5.0 MAT-file, written by Octave 3.8.0, 2014-12-18 10:48:09 UTC',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[ 0.00491231,  0.69319077, -0.20364049, ...,  0.23149795,
         -0.28978574, -0.49329397],
        [ 0.11072935, -0.07990259, -0.20364049, ...,  0.09356344,
         -0.25638541, -0.49329397],
        [ 0.21654639, -0.27244466, -0.20364049, ...,  0.02459619,
         -0.25638541,  1.14001753],
        ...,
        [-0.41835583, -0.91998844, -0.16463485, ..., -1.49268341,
          0.24461959, -0.49329397],
        [-0.41835583, -0.91998844, -0.15093411, ..., -1.42371616,
          0.14441859, -0.49329397],
        [-0.41835583, -0.91998844, -0.20364049, ..., -1.28578165,
          3.58465295, -0.49329397]]),
 'y': array([[0.],
        [0.],
        [0.],
        ...,
        [1.],
        [1.],
        [1.]])}

In [46]:
len(data)

5

In [47]:
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'X', 'y'])

In [48]:
data.values()

dict_values([b'MATLAB 5.0 MAT-file, written by Octave 3.8.0, 2014-12-18 10:48:09 UTC', '1.0', [], array([[ 0.00491231,  0.69319077, -0.20364049, ...,  0.23149795,
        -0.28978574, -0.49329397],
       [ 0.11072935, -0.07990259, -0.20364049, ...,  0.09356344,
        -0.25638541, -0.49329397],
       [ 0.21654639, -0.27244466, -0.20364049, ...,  0.02459619,
        -0.25638541,  1.14001753],
       ...,
       [-0.41835583, -0.91998844, -0.16463485, ..., -1.49268341,
         0.24461959, -0.49329397],
       [-0.41835583, -0.91998844, -0.15093411, ..., -1.42371616,
         0.14441859, -0.49329397],
       [-0.41835583, -0.91998844, -0.20364049, ..., -1.28578165,
         3.58465295, -0.49329397]]), array([[0.],
       [0.],
       [0.],
       ...,
       [1.],
       [1.],
       [1.]])])

### Input Feature Shape in Mat file format

In [49]:
type(data['X']),data['X'].shape

(numpy.ndarray, (1831, 21))

### Output Feature Shape in Mat file format

In [50]:
type(data['y']),data['y'].shape

(numpy.ndarray, (1831, 1))

In [51]:
df_columns=['Data','#Samples','#Dimension','Outlier Perc',
        'ABOD','LOF','CBLOF','FB','HBOS','IForest',
       'KNN','MCD','OCSVM','PCA']

### ROC(Region of characterstics) Performance evolution table

In [52]:
roc_df=pd.DataFrame(columns=df_columns)

In [53]:
roc_df

,Data,#Samples,#Dimension,Outlier Perc,ABOD,LOF,CBLOF,FB,HBOS,IForest,KNN,MCD,OCSVM,PCA


### precision_n_scores Perfrmance Evolution Table

In [54]:
prn_df=pd.DataFrame(columns=df_columns)
prn_df

,Data,#Samples,#Dimension,Outlier Perc,ABOD,LOF,CBLOF,FB,HBOS,IForest,KNN,MCD,OCSVM,PCA


### Time dataframe

In [55]:

time_df=pd.DataFrame(columns=df_columns)
time_df

,Data,#Samples,#Dimension,Outlier Perc,ABOD,LOF,CBLOF,FB,HBOS,IForest,KNN,MCD,OCSVM,PCA


### Exploaring all mat files

In [58]:
from time import time
random_state = np.random.RandomState(42)

for mat_file in mat_file_list:
    print("\n",mat_file)
    mat=loadmat(os.path.join(mat_file))
    X=mat['X']
    y=np.ravel(mat['y']) #method to convert 2d into 1d
    outliers_fraction = np.count_nonzero(y) / len(y)
    outliers_percentage=round(outliers_fraction*100,ndigits=4)
    
    roc_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
    prn_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
    time_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
    
    X_train, X_test, y_train, y_test= train_test_split(X,y,test_size=0.4,random_state=random_state)
    X_train_norm, X_test_norm = standardizer(X_train, X_test)
    classifiers= {'Angle-based Outlier Detector (ABOD)': ABOD(contamination=outliers_fraction),
                 'Cluster-based Local Outlier Factor':CBLOF(contamination=outliers_fraction, check_estimator= False, random_state=random_state),
                 'Feature Bagging': FeatureBagging(contamination=outliers_fraction, random_state=random_state),
                 'Histogram-base Outlier Detection (HBOS)': HBOS(contamination=outliers_fraction),
                 'Isolation Forest': IForest(contamination=outliers_fraction, random_state=random_state),
                 'K Nearest Neighbors(KNN)': KNN(contamination=outliers_fraction),
                 'Local Outlier Factor(LOF)': LOF(contamination=outliers_fraction),
                 'Minimum Covariance Determinant (MCD)': MCD(contamination=outliers_fraction, random_state=random_state),
                 'One-class SVM (OCSVM)': OCSVM(contamination=outliers_fraction),
                 'Principal Component Analysis (PCA)': PCA(contamination=outliers_fraction, random_state=random_state)}
    for clf_name, clf in classifiers.items():
        t0 = time()
        clf.fit(X_train_norm)
        test_scores = clf.decision_function(X_test_norm)
        t1 = time()
        duration = round(t1- t0, ndigits=4)
        time_list.append(duration)
        
        roc= round(roc_auc_score(y_test, test_scores), ndigits=4)
        prn = round(precision_n_scores(y_test, test_scores), ndigits=4)
        print('{clf_name} ROC:{roc}, precision @ rank n:{prn},''execution time: {duration}s'.format(clf_name=clf_name, roc=roc, prn=prn ,duration=duration))
        
        roc_list.append(roc)
        prn_list.append(prn)
        
    temp_df = pd.DataFrame(time_list).transpose()
    temp_df.columns = df_columns
    time_df = pd.concat([time_df, temp_df], axis=0)
        
    temp_df = pd.DataFrame(roc_list).transpose()
    temp_df.columns = df_columns
    roc_df = pd.concat([roc_df, temp_df], axis=0)
        
    temp_df = pd.DataFrame(prn_list).transpose()
    temp_df.columns = df_columns
    prn_df = pd.concat([prn_df, temp_df], axis=0)
        


 arrhythmia.mat
Angle-based Outlier Detector (ABOD) ROC:0.7687, precision @ rank n:0.3571,execution time: 0.245s


C:\Users\User\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:973: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  warnings.warn("'n_jobs' was deprecated in version 0.23 and will be"


Cluster-based Local Outlier Factor ROC:0.7684, precision @ rank n:0.4643,execution time: 3.3912s
Feature Bagging ROC:0.7799, precision @ rank n:0.5,execution time: 0.663s
Histogram-base Outlier Detection (HBOS) ROC:0.8511, precision @ rank n:0.5714,execution time: 2.4361s
Isolation Forest ROC:0.8478, precision @ rank n:0.5357,execution time: 0.675s
K Nearest Neighbors(KNN) ROC:0.782, precision @ rank n:0.5,execution time: 0.101s
Local Outlier Factor(LOF) ROC:0.7787, precision @ rank n:0.4643,execution time: 0.078s


C:\Users\User\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:647: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "


Minimum Covariance Determinant (MCD) ROC:0.8228, precision @ rank n:0.4286,execution time: 2.0681s
One-class SVM (OCSVM) ROC:0.7986, precision @ rank n:0.5,execution time: 0.069s
Principal Component Analysis (PCA) ROC:0.7997, precision @ rank n:0.5,execution time: 0.182s

 cardio.mat
Angle-based Outlier Detector (ABOD) ROC:0.5763, precision @ rank n:0.1875,execution time: 0.694s
Cluster-based Local Outlier Factor ROC:0.8221, precision @ rank n:0.4844,execution time: 0.164s


...

Isolation Forest ROC:0.3539, precision @ rank n:0.0,execution time: 0.469s
K Nearest Neighbors(KNN) ROC:0.3562, precision @ rank n:0.0,execution time: 0.015s
Local Outlier Factor(LOF) ROC:0.3562, precision @ rank n:0.0,execution time: 0.005s
Minimum Covariance Determinant (MCD) ROC:0.4203, precision @ rank n:0.0,execution time: 0.077s
One-class SVM (OCSVM) ROC:0.4078, precision @ rank n:0.0,execution time: 0.002s
Principal Component Analysis (PCA) ROC:0.3633, precision @ rank n:0.0625,execution time: 0.002s

 vowels.mat
Angle-based Outlier Detector (ABOD) ROC:0.8982, precision @ rank n:0.2857,execution time: 0.5364s
Cluster-based Local Outlier Factor ROC:0.8608, precision @ rank n:0.2143,execution time: 0.118s


C:\Users\User\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:973: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  warnings.warn("'n_jobs' was deprecated in version 0.23 and will be"


Feature Bagging ROC:0.9124, precision @ rank n:0.2143,execution time: 0.4s
Histogram-base Outlier Detection (HBOS) ROC:0.7567, precision @ rank n:0.2143,execution time: 0.006s
Isolation Forest ROC:0.7758, precision @ rank n:0.2143,execution time: 0.5634s
K Nearest Neighbors(KNN) ROC:0.9746, precision @ rank n:0.3571,execution time: 0.111s
Local Outlier Factor(LOF) ROC:0.9192, precision @ rank n:0.2857,execution time: 0.044s
Minimum Covariance Determinant (MCD) ROC:0.6966, precision @ rank n:0.0714,execution time: 1.1649s
One-class SVM (OCSVM) ROC:0.8426, precision @ rank n:0.2857,execution time: 0.056s
Principal Component Analysis (PCA) ROC:0.6907, precision @ rank n:0.2143,execution time: 0.01s

 wbc.mat
Angle-based Outlier Detector (ABOD) ROC:0.9573, precision @ rank n:0.5556,execution time: 0.159s
Cluster-based Local Outlier Factor ROC:0.9751, precision @ rank n:0.6667,execution time: 0.0772s
Feature Bagging ROC:0.979, precision @ rank n:0.7778,execution time: 0.0976s
Histogram-base

C:\Users\User\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:973: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  warnings.warn("'n_jobs' was deprecated in version 0.23 and will be"


Isolation Forest ROC:0.9674, precision @ rank n:0.5556,execution time: 0.491s
K Nearest Neighbors(KNN) ROC:0.9713, precision @ rank n:0.4444,execution time: 0.028s
Local Outlier Factor(LOF) ROC:0.979, precision @ rank n:0.6667,execution time: 0.01s
Minimum Covariance Determinant (MCD) ROC:0.9658, precision @ rank n:0.5556,execution time: 0.102s
One-class SVM (OCSVM) ROC:0.9782, precision @ rank n:0.6667,execution time: 0.006s
Principal Component Analysis (PCA) ROC:0.9775, precision @ rank n:0.6667,execution time: 0.006s


In [59]:
roc_df

,Data,#Samples,#Dimension,Outlier Perc,ABOD,LOF,CBLOF,FB,HBOS,IForest,KNN,MCD,OCSVM,PCA
0,arrhythmia,452,274,14.6018,0.7687,0.7684,0.7799,0.8511,0.8478,0.782,0.7787,0.8228,0.7986,0.7997
0,cardio,1831,21,9.6122,0.5763,0.8221,0.4879,0.8453,0.9316,0.6959,0.4715,0.8781,0.9507,0.9638
0,glass,214,9,4.2056,0.7104,0.8506,0.7043,0.6524,0.7195,0.7805,0.7774,0.7165,0.6189,0.622
0,ionosphere,351,33,35.8974,0.9004,0.8952,0.8933,0.5195,0.8294,0.9134,0.8989,0.9399,0.8372,0.7971
0,letter,1600,32,6.25,0.8465,0.7423,0.866,0.5728,0.5836,0.845,0.8409,0.7499,0.5744,0.48
0,lympho,148,18,4.0541,0.9382,0.9709,0.9673,0.9964,0.9855,0.9636,0.9636,0.9164,0.9636,0.9818
0,mnist,7603,100,9.2069,0.7813,0.8447,0.7259,0.5675,0.7813,0.8409,0.7085,0.863,0.8417,0.8396
0,musk,3062,166,3.1679,0.0809,1,0.5228,0.9999,0.9992,0.7348,0.5323,1,1,1
0,optdigits,5216,64,2.8758,0.4428,0.7852,0.4641,0.8822,0.5442,0.3824,0.4584,0.3486,0.4972,0.504
0,pendigits,6870,16,2.2707,0.7008,0.9609,0.4687,0.9294,0.9482,0.7602,0.481,0.8271,0.93,0.9332


In [60]:
time_df

,Data,#Samples,#Dimension,Outlier Perc,ABOD,LOF,CBLOF,FB,HBOS,IForest,KNN,MCD,OCSVM,PCA
0,arrhythmia,452,274,14.6018,0.245,3.3912,0.663,2.4361,0.675,0.101,0.078,2.0681,0.069,0.182
0,cardio,1831,21,9.6122,0.694,0.164,0.9821,0.01,0.585,0.208,0.1138,0.7384,0.1092,0.1248
0,glass,214,9,4.2056,0.0582,0.0468,0.0572,0,0.4732,0.014,0.004,0.067,0.002,0.003
0,ionosphere,351,33,35.8974,0.135,0.067,0.0764,0,0.4656,0.0416,0.008,0.127,0.007,0.015
0,letter,1600,32,6.25,0.653,0.1248,1.0478,0.017,0.7144,0.216,0.114,1.7418,0.094,0.007
0,lympho,148,18,4.0541,0.055,0.076,0.035,0.0156,0.4472,0.01,0.004,0.055,0.002,0.003
0,mnist,7603,100,9.2069,9.6814,1.1107,106.075,0.158,5.3253,15.1329,13.5348,11.8797,8.9385,1.0481
0,musk,3062,166,3.1679,5.0823,0.9331,26.2975,0.191,3.5662,4.0122,3.3912,47.1607,1.9541,0.284
0,optdigits,5216,64,2.8758,5.7003,0.9311,24.1864,0.065,2.1341,3.4492,2.3801,3.1122,1.9821,0.096
0,pendigits,6870,16,2.2707,3.3592,0.453,7.3624,0.02,1.4301,1.1161,0.883,4.7353,1.5091,0.013


In [61]:
prn_df

,Data,#Samples,#Dimension,Outlier Perc,ABOD,LOF,CBLOF,FB,HBOS,IForest,KNN,MCD,OCSVM,PCA
0,arrhythmia,452,274,14.6018,0.3571,0.4643,0.5,0.5714,0.5357,0.5,0.4643,0.4286,0.5,0.5
0,cardio,1831,21,9.6122,0.1875,0.4844,0.1406,0.4688,0.4531,0.2812,0.125,0.3906,0.5938,0.6875
0,glass,214,9,4.2056,0.25,0.25,0.25,0,0.25,0.25,0.25,0,0.25,0.25
0,ionosphere,351,33,35.8974,0.8214,0.8036,0.75,0.3393,0.6607,0.8393,0.75,0.8571,0.7143,0.5893
0,letter,1600,32,6.25,0.275,0.175,0.4,0.125,0.05,0.3,0.325,0.075,0.1,0.05
0,lympho,148,18,4.0541,0.4,0.6,0.6,0.8,0.6,0.6,0.6,0.6,0.6,0.8
0,mnist,7603,100,9.2069,0.3562,0.4007,0.3664,0.1199,0.3116,0.4144,0.339,0.3973,0.3801,0.3767
0,musk,3062,166,3.1679,0.0333,1,0.1667,0.9667,0.9,0.2333,0.1333,0.9667,1,1
0,optdigits,5216,64,2.8758,0.0161,0,0.0484,0.2581,0.0161,0,0.0484,0,0,0
0,pendigits,6870,16,2.2707,0.0308,0.3077,0.0462,0.2615,0.2615,0.0462,0.0462,0.0615,0.2923,0.3385
